Set your Pulse API key as an environment variable:

```bash
export PULSE_API_KEY="YOUR_API_KEY"
```

In [ ]:
# Install qubex library if not installed
# !pip install git+https://github.com/amachino/qubex.git

In [ ]:
import numpy as np
import qubex as qx

# Import the PulseAPI class from the qubex library
from qubex.api import PulseAPI

In [ ]:
# Create an instance of the PulseAPI class
api = PulseAPI(
    chip_id="64Q",
    qubits=[
        "Q60",
        "Q61",
        "Q62",
        "Q63",
    ],
    # api_key="YOUR_API_KEY",  # Set if you can't use an environment variable
)

In [ ]:
# Print the parameters of the control device
api.params

In [ ]:
# Print the available targets
api.targets

In [ ]:
waveform = [0.01 + 0.01j] * 5

# Measure the qubits with average mode
result = api.measure(
    {
        "Q60": waveform,
        "Q61": waveform,
        "Q62": waveform,
        "Q63": waveform,
    },
    mode="avg",
    shots=1024,
    interval=150 * 1024,
)

In [ ]:
# Plot the averaged result
result.plot()

# Print the averaged data
result.data

In [ ]:
# Measure the qubits with singleshot mode
result = api.measure(
    {
        "Q60": waveform,
        "Q61": waveform,
        "Q62": waveform,
        "Q63": waveform,
    },
    mode="single",
    shots=1024,
    interval=150 * 1024,
)

In [ ]:
# Plot the singleshot result
result.plot()

# Print the singleshot data
result.data

In [ ]:
# Waveform as a list of complex numbers
iq_list = [0.01 + 0.01j] * 5

# Pulse object from qubex.pulse
pulse = qx.pulse.Gaussian(
    duration=60,
    amplitude=0.1,
    sigma=10,
)

# Plot the pulse
pulse.plot()

# You can pass the waveform as a list, numpy array or Pulse object of qubex.pulse
result = api.measure(
    {
        "Q60": iq_list,  # list[complex]
        "Q61": np.array(iq_list),  # np.ndarray
        "Q62": qx.Pulse(iq_list),  # qubex.pulse.Pulse
        "Q63": pulse,  # qubex.pulse.Gaussian
    },
)

In [ ]:
# Use PulseSchedule to compose a complex pulse schedule
with qx.PulseSchedule(api.qubits) as ps:
    ps.add("Q60", pulse)
    ps.barrier()
    ps.add("Q61", pulse)

# Plot the pulse schedule
ps.plot()

# Measure the qubits with the pulse schedule
result = api.measure(ps)

# Plot the result
result.plot()

In [ ]:
from collections import defaultdict

from qubex.analysis import IQPlotter

plotter = IQPlotter()
results = defaultdict(list)

# Rabi experiment
for T in np.arange(0, 200, 20):
    result = api.measure(
        {qubit: qx.pulse.Rect(duration=T, amplitude=0.03) for qubit in api.qubits}
    )
    for qubit in api.qubits:
        results[qubit].append(result.data[qubit].kerneled)
    plotter.update(results)